# ZILLOW

# Acquire

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from env import host, user, password

In [ ]:
# Creating a string that connects me to MySQLWorkbench

def get_connection(db, user=user, host=host, password=password):
    '''
    get_connection uses login info from env.py file to access Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    

In [ ]:
# Getting the data from the Codeup database

def get_zillow_data():
    '''
    get_zillow_data() gets the zillow (only properties_2017 table) data from Codeup db, then writes it to a csv file,
    and returns the DF.
    '''
    # Creating a SQL query
    sql_query = '''
                SELECT DISTINCT *,
                    pred17.logerror,
                    pred17.transactiondate,
                    acd.airconditioningdesc,
                    asd.architecturalstyledesc,
                    bcd.buildingclassdesc,
                    hsd.heatingorsystemdesc,
                    sd.storydesc,
                    tcd.typeconstructiondesc
                FROM properties_2017 AS p17
                LEFT JOIN predictions_2017 AS pred17 USING (parcelid)
                LEFT JOIN propertylandusetype USING (propertylandusetypeid)
                LEFT JOIN airconditioningtype AS acd USING (airconditioningtypeid)
                LEFT JOIN architecturalstyletype AS asd USING (architecturalstyletypeid)
                LEFT JOIN buildingclasstype AS bcd USING (buildingclasstypeid)
                LEFT JOIN heatingorsystemtype AS hsd USING (heatingorsystemtypeid)
                LEFT JOIN storytype AS sd USING (storytypeid)
                LEFT JOIN typeconstructiontype AS tcd USING (typeconstructiontypeid)
                JOIN (SELECT MAX(transactiondate BETWEEN '2017-01-01' AND '2017-12-31') AS max_sale_date
                     FROM predictions_2017 AS pred17) AS pred17
                WHERE propertylandusedesc = 'Single Family Residential'
                AND longitude IS NOT NULL
                AND latitude IS NOT NULL;
                '''
    
    # Reading in the DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    return df

In [ ]:
# Assigning the dataframe to a variable and displaying of it to have a first look

houses = get_zillow_data()
houses.head()

In [ ]:
# Displaying number of rows and columns

houses.shape

In [ ]:
# Displaying some general information about the data

houses.info()

In [ ]:
# Writing houses to csv on my computer

houses.to_csv('houses.csv')

In [ ]:
# Displaying a first statistical analysis of the data through .describe()

houses.describe().T

In [ ]:
# Displaying statistical analysis of the data through.descibe()
# with astype('int64') to cut through all the noise of the float data

houses.describe().T.astype('int64')

In [ ]:
houses.head()

In [ ]:
# Checking for duplicates

houses.columns.duplicated().any()

In [ ]:
# Dropping the duplicates

houses.drop_duplicates(keep=False)

In [ ]:
# Checking to see the number of columns left

houses.shape

In [ ]:
# Running a list of the columns to detect the duplicates

houses.columns.tolist()

**The duplicates are 2 id columns. They are not essential to the data exploration. I will drop them along with other id columns**

**However in case one of the columns are necessary in a different case, I have put together a function that rename the columns in order to differentiate them**

In [ ]:
# # For loop to rename duplicate columns (particularly if they have the same name and not the same content)

# cols=pd.Series(df.columns)
# for dup in df.columns[df.columns.duplicated(keep=False)]: 
#     cols[df.columns.get_loc(dup)] = ([dup + '_' + str(d_idx) 
#                                      if d_idx != 0 
#                                      else dup 
#                                      for d_idx in range(df.columns.get_loc(dup).sum())]
#                                     )
# df.columns=cols

In [ ]:
# Dropping id columns

houses = houses.drop(['typeconstructiontypeid',
 'storytypeid',
 'heatingorsystemtypeid',
 'architecturalstyletypeid',
 'airconditioningtypeid',
 'propertylandusetypeid',
 'id',
 'buildingqualitytypeid',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'decktypeid'], axis = 1)
houses.columns.tolist()

In [ ]:
houses.shape

In [ ]:
# A loop to print out the value_counts of the columns

for column in houses.columns:
    print(column)
    print(houses[column].value_counts())
    print('\n##########################\n')

In [ ]:
# # This code allows me to display the count of nulls in each column mentioned
# # Still need to shape it into a function or a loop

# houses[['bedroomcnt',
#  'calculatedbathnbr',
#  'finishedfloor1squarefeet',
#  'calculatedfinishedsquarefeet',
#  'finishedsquarefeet12',
#  'finishedsquarefeet15',
#  'finishedsquarefeet50',
#  'finishedsquarefeet6']].isna().sum().reset_index(name="n").plot.bar(x='index', y='n', rot=75)

In [ ]:
# Trying my function

def multi_frequency(df,vars):
    '''multi_frequency takes a dataframe in *arg and a *kwarg in the form of a list of columns
    and return a dataframe with the count and the frequency of the data
    '''
    frequency=df[vars].isnull().sum()
    percentage=df[vars].isnull().sum()*100/(len(df))
    df=pd.concat([frequency,percentage], axis=1, keys=['num_rows_missing', 'pct_rows_missing'])
    return df
multi_frequency(houses, ['latitude'])

In [ ]:
# Displaying the columns and their the count and percent of missing rows ordered by percent in a descending order

multi_frequency(houses, houses.columns).sort_values(by='pct_rows_missing', ascending=False)

In [ ]:
# def multi_frequency_rows(df,vars):
#     '''multi_frequency takes a dataframe in *arg and a *kwarg in the form of a list of columns
#     and return a dataframe with the count and the frequency of the data
#     '''
#     frequency=df[vars].isnull().sum(axis=1)
#     percentage=df[vars].isnull().sum(axis=1)*100/(len(df))
#     df=pd.concat([frequency,percentage], axis=1, keys=['num_entries_missing', 'pct_entries_missing'])
#     return df
# multi_frequency(houses, ['basementsqft'])

In [ ]:
# Assigning the table above to a variable and selecting a cut off for the percentage of rows missing to eliminate

frequency = multi_frequency(houses, houses.columns).sort_values(by='pct_rows_missing', ascending=False)
frequency.pct_rows_missing >= 33.589411

In [ ]:
# Percent of columns that is missing +30% of its entries

percent_nullvalues_columns = (len(frequency[frequency.pct_rows_missing >= 33.589411]))/len(frequency)

In [ ]:
percent_nullvalues_columns

## Takeaways
- 54.71% of the data is missing between 33% and 100% of its values
- This makes me think that the best cut off would be the 33% threshold
- Many columns are id columns and should be removed. They have no impact on the analysis of the data and teh dataframe already has parcelid as unique identifier. Here is the list of the columns to drop:
     'typeconstructiontypeid',
     'storytypeid',
     'heatingorsystemtypeid',
     'buildingclasstypeid',
     'architecturalstyletypeid',
     'airconditioningtypeid',
     'propertylandusetypeid',
     'id',
     'buildingqualitytypeid',
     'id_1'

# Prepare

## 1. Remove any properties that are likely to be something other than single unit properties.

**Single unit properties were selected in the SQL query. The code for it is shown in the SQL query**

## 2. Create a function that will drop rows or columns based on the percent of values that are missing

In [ ]:
houses.head()

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A. Handling missing values

In [ ]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    handle_missing_values calculates the number of recquired columns and rows
    based on an arbitrary prop_required_row/column float times the number
    of columns/rows respectively as ordered in the function
    then uses the 'thresh' argument to apply that number to the dropna function
    '''
    required_columns = df.shape[0] * prop_required_column
    required_rows = df.shape[1] * prop_required_row
    
    df = df.dropna(axis=0, thresh = required_rows)
    df = df.dropna(axis=1, thresh = required_columns)
    return df

In [ ]:
houses = handle_missing_values(houses, prop_required_column=.7, prop_required_row=.7) 
houses.head()

In [ ]:
def examine_and_prepare()

In [ ]:
def _maybe_dedup_names(self, names):
    # see gh-7160 and gh-9424: this helps to provide
    # immediate alleviation of the duplicate names
    # issue and appears to be satisfactory to users,
    # but ultimately, not needing to butcher the names
    # would be nice!
    if self.mangle_dupe_cols:
        names = list(names)  # so we can index
        counts = {}

        for i, col in enumerate(names):
            cur_count = counts.get(col, 0)

            if cur_count > 0:
                names[i] = '%s.%d' % (col, cur_count)

            counts[col] = cur_count + 1

    return names

In [ ]:
figure, ax1 = plt.subplots()
ax1.plot(df.iloc[:,0],df.iloc[:,1],linewidth=0.5,zorder=1, label = )
ax1.plot(df.iloc[:,0],df.iloc[:,2],linewidth=0.5,zorder=1, label = )